# Kaggle Competition: Patfinder Pawpularity score

## 1. CNN as a feature extractor

Import libraries

In [1]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timm
from torchvision import transforms
import seaborn as sns
import warnings
from tqdm import tqdm

#Backgrounds configurations
sns.set_theme()
warnings.filterwarnings("ignore")

Device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Parameters and Hyperparameters

In [3]:
# Path control
path = '../input/petfinder-pawpularity-score/'
img_dir = path + 'train/'
label_path = path + 'train.csv'

# Parameters
random_seed = 17
val_size = 0.2
img_size = 224

#Hiperparameters
NUM_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.0001
MOMENTUM = 0.9
F_OPTIMIZER = optim.SGD
LOSS_FUNCTION = nn.MSELoss()
MODEL_NAME='tf_efficientnet_b0_ns'

# To make transformations of the images
transform = transforms.Compose(
                   [
                    transforms.ToTensor(),
                    transforms.Resize((img_size,img_size)),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])



Setting randomness

In [4]:
def seed_torch(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=random_seed)

Create Pytorch Dataset class

In [5]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dataframe, img_name, label_name, transform=None):
        self.img_dir = img_dir
        self.img_label = label_dataframe
        self.transform = transform
        self.img_name = img_name
        self.label_name = label_name
        
    def __len__(self):
        return len(self.img_label)

    def __getitem__(self, ix):
        img_path = self.img_dir + self.img_label[self.img_name][ix] + '.jpg'
        X_ix = plt.imread(img_path)
        Y_ix = self.img_label[self.label_name][ix]
        Y_ix = Y_ix.astype(np.float32)
        if self.transform:
            X_ix = self.transform(X_ix)
        return X_ix, Y_ix
     

Instance the dataset class: train and validation instances

In [6]:
df = pd.read_csv(label_path)
df['nPawpularity']=(df['Pawpularity'] - df['Pawpularity'].min()) / (df['Pawpularity'].max() - df['Pawpularity'].min())
df_train, df_val = train_test_split(df, test_size=0.2, random_state=random_seed)
df_train.reset_index(inplace=True)
df_val.reset_index(inplace=True)

dataset = {
    'train': Dataset(img_dir, df_train, 'Id', 'Pawpularity', transform=transform),
    'val'  : Dataset(img_dir, df_val  , 'Id', 'Pawpularity', transform=transform)
}

Create Pytorch Dataloader

In [7]:
dataloader = {
    'train': torch.utils.data.DataLoader(dataset['train'], batch_size=BATCH_SIZE, shuffle=True),
    'val'  : torch.utils.data.DataLoader(dataset['val']  , batch_size=BATCH_SIZE, shuffle=True)
}

Defining the model (CNN)

In [8]:
'''class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(44944, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
net.to(device)  '''      

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=44944, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=1, bias=True)
)

In [9]:
class Net_pretrained(nn.Module):
    def __init__(self, timm_model_name, target_size, pretrained=True):
        super().__init__()
        self.model = timm.create_model(timm_model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(self.n_features, target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature

    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output


net = Net_pretrained(timm_model_name=MODEL_NAME, target_size=1, pretrained=True)
net.to(device)  

Net_pretrained(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_r

Prueba con un pase forward del modelo con un ejemplo random

In [10]:
tensordeprueba = torch.rand(5, 3, 224, 224).to(device)
net(tensordeprueba)

tensor([[ 0.0150],
        [-0.0192],
        [-0.0560],
        [-0.0271],
        [ 0.1099]], device='cuda:0', grad_fn=<AddmmBackward>)

Train the model

In [ ]:
#Easy training loop
'''# Define Loss function & Optimizer
criterion = LOSS_FUNCTION
optimizer = F_OPTIMIZER(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

# Training loop
for epoch in range(NUM_EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader['train']):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 0:    # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')'''

In [11]:
def fit(model, dataloader, epochs=NUM_EPOCHS):
    criterion = LOSS_FUNCTION
    optimizer = F_OPTIMIZER(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            # get the inputs; data is a list of [inputs, labels]
            X, y = batch
            X, y = X.to(device), y.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()
            
            # forward + backward + optimize
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()

            # save loss
            train_loss.append(loss.item())
            bar.set_description(f"loss {np.mean(train_loss):.5f} rmse {np.mean(train_loss)**0.5:.5f}")

        bar = tqdm(dataloader['val'])
        val_loss = []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                # get the inputs; data is a list of [inputs, labels]
                X, y = batch
                X, y = X.to(device), y.to(device)

                # forward
                y_hat = model(X)
                loss = criterion(y_hat, y)

                # save loss
                val_loss.append(loss.item())
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_rmse {np.mean(val_loss)**0.5:.5f}")

        print(f"::::Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} rmse {np.mean(train_loss)**0.5:.5f} val_rmse {np.mean(val_loss)**0.5:.5f}::::")

fit(net, dataloader, epochs=2)

  0%|          | 0/124 [00:00<?, ?it/s]

::::Epoch 1/2 loss 662.68705 val_loss 413.34349 rmse 25.74271 val_rmse 20.33085::::


val_loss 411.93023 val_rmse 20.29606: 100%|██████████| 31/31 [00:42<00:00,  1.38s/it]

::::Epoch 2/2 loss 431.89580 val_loss 411.93023 rmse 20.78210 val_rmse 20.29606::::


Save the model

In [12]:
torch.save(net.state_dict(), '../models/effnetb0_bs64_lr00001_m09.pt')

In [20]:
net(tensordeprueba)

tensor([[12.1433],
        [11.7672],
        [12.4146],
        [11.5555],
        [11.5933]], device='cuda:0', grad_fn=<AddmmBackward>)

In [23]:
torch.save(tensordeprueba, '../others/tensordeprueba.pt')

Pendiente coger partes de codigo interesantes de este training loop

In [ ]:
def fit(model, dataloader, scheduler=None, epochs=NUM_EPOCHS, log_each=1, weight_decay=0, early_stopping=0, verbose=1):
    criterion = LOSS_FUNCTION
    optimizer = F_OPTIMIZER(net.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM)
        
    l, acc, lr = [], [], []
    val_l, val_acc = [], []
    best_acc, step = 0, 0
    for e in range(1, epochs+1):
        _l, _acc = [], []
        for param_group in optimizer.param_groups:
            lr.append(param_group['lr'])
        model.train()
        for x_b, y_b in dataloader['train']:
            optimizer.zero_grad()
            y_pred = model(x_b)
            loss = criterion(y_pred, y_b)
            _l.append(loss.item())
            
            loss.backward()
            optimizer.step()
            _acc.append(accuracy_score(y_b.cpu().numpy(), y_probas.cpu().detach().numpy()))
        l.append(np.mean(_l))
        acc.append(np.mean(_acc))
        model.eval()
        _l, _acc = [], []
        with torch.no_grad():
            for x_b, y_b in dataloader['val']:
                y_pred = model(x_b)
                loss = criterion(y_pred, y_b)
                _l.append(loss.item())
                y_probas = torch.argmax(softmax(y_pred), axis=1)
                _acc.append(accuracy_score(y_b.cpu().numpy(), y_probas.cpu().numpy()))
        val_l.append(np.mean(_l))
        val_acc.append(np.mean(_acc))
        # guardar mejor modelo
        if val_acc[-1] > best_acc:
            best_acc = val_acc[-1]
            torch.save(model.state_dict(), 'ckpt.pt')
            step = 0
            if verbose == 2:
                print(f"Mejor modelo guardado con acc {best_acc:.5f} en epoch {e}")
        step += 1
        if scheduler:
            scheduler.step()
        # parar
        if early_stopping and step > early_stopping:
            print(f"Entrenamiento detenido en epoch {e} por no mejorar en {early_stopping} epochs seguidas")
            break
        if not e % log_each and verbose:
            print(f"Epoch {e}/{epochs} loss {l[-1]:.5f} acc {acc[-1]:.5f} val_loss {val_l[-1]:.5f} val_acc {val_acc[-1]:.5f} lr {lr[-1]:.5f}")
    # cargar mejor modelo
    model.load_state_dict(torch.load('ckpt.pt'))
    return {'epoch': list(range(1, len(l)+1)), 'loss': l, 'acc': acc, 'val_loss': val_l, 'val_acc': val_acc, 'lr': lr}